In [231]:
import os
os.path.abspath("./RadLex")

'/home/yuxiangliao/PhD/RadLex'

In [232]:
import os, sys
from loguru import logger

LOG_ROOT = os.path.abspath("./")
LOG_FILE = LOG_ROOT + "/logs/radlex.log"

# Remove all handlers and reset stderr
logger.remove(handler_id=None)
logger.add(
    LOG_FILE,
    level="TRACE",
    mode="w",
    backtrace=False,
    diagnose=True,
    colorize=False,
    format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
)
logger.info("\r\n" + ">" * 29 + "\r\n" + ">>> New execution started >>>" + "\r\n" + ">" * 29)
# To filter log level: TRACE=5, DEBUG=10, INFO=20, SUCCESS=25, WARNING=30, ERROR=40, CRITICAL=50
logger.add(sys.stdout, level="INFO", filter=lambda record: record["level"].no < 40, colorize=True)
logger.add(sys.stderr, level="ERROR", backtrace=False, diagnose=True, colorize=True)

9

In [13]:
with open("/home/yuxiangliao/PhD/RadLex/owlapi.xrdf", "r") as f:
  radlex = f.readlines()
    
print(len(radlex))

1499968


In [32]:
# import xml.etree.cElementTree as ET  
import xml.etree.ElementTree as ET 
tree = ET.parse("/home/yuxiangliao/PhD/RadLex/owlapi.xrdf")
root = tree.getroot()

# Analyse the content

In [72]:
child_lv1 = set()
for child in root:
    child_lv1.add(child.tag)
for nodeName in child_lv1:
    print(f"node:{nodeName}, len: {len(root.findall(nodeName))}")

node:{http://www.w3.org/2002/07/owl#}Class, len: 46761
node:{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description, len: 24076
node:{http://www.w3.org/2002/07/owl#}ObjectProperty, len: 52
node:{http://www.w3.org/2002/07/owl#}AnnotationProperty, len: 30
node:{http://www.w3.org/2002/07/owl#}NamedIndividual, len: 24075
node:{http://www.w3.org/2002/07/owl#}Ontology, len: 1
node:{http://www.w3.org/2000/01/rdf-schema#}Datatype, len: 1
node:{http://www.w3.org/2002/07/owl#}Axiom, len: 7316


In [182]:
# All the XML tags under specific path
classes = root.findall("{http://www.w3.org/2002/07/owl#}Class")

class_tags = set()
for class_node in classes:
    for ele in class_node.iter():
        class_tags.add(ele.tag)
        
for i in class_tags:
    print(i)

{http://radlex.org/RID/}JHU_White-Matter_Tractography_Atlas
{http://radlex.org/RID/}Talairach
{http://radlex.org/RID/}Preferred_name
{http://www.w3.org/2002/07/owl#}onProperty
{http://www.w3.org/2000/01/rdf-schema#}subClassOf
{http://radlex.org/RID/}Synonym_German
{http://radlex.org/RID/}JHU_DTI-81
{http://www.w3.org/2002/07/owl#}Class
{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description
{http://radlex.org/RID/}Freesurfer
{http://radlex.org/RID/}Preferred_name_German
{http://radlex.org/RID/}Acronym
{http://www.w3.org/2002/07/owl#}unionOf
{http://radlex.org/RID/}Radlex_version_of_class_change
{http://radlex.org/RID/}SNOMED_ID
{http://www.w3.org/2002/07/owl#}Restriction
{http://www.w3.org/2000/01/rdf-schema#}label
{http://radlex.org/RID/}Preferred_Name_for_Obsolete
{http://radlex.org/RID/}FMAID
{http://radlex.org/RID/}Misspelling_of_term
{http://radlex.org/RID/}Comment
{http://radlex.org/RID/}Synonym
{http://radlex.org/RID/}Replaced_by
{http://radlex.org/RID/}Definition
{http://www.w

In [ ]:
idx1 = 0
idx2 = 1
node = root.find("{http://www.w3.org/2000/01/rdf-schema#}Datatype")
print(node)
print(node[0].tag)
print(node[0].attrib)
print(f"text:{node[0].text}")
print(bool(node[0].attrib))
print(len(node[0].attrib))

if not node[0].attrib:
    print(1)

# Extract

In [142]:
import pandas as pd

## A set of relationships

In [215]:
object_properties = root.findall("{http://www.w3.org/2002/07/owl#}ObjectProperty")
df_relationships = pd.DataFrame(columns=['relationshipName','subPropertyOf','inverseOf'])

for objProp in object_properties:
    df_relationships.loc[len(df_relationships),'relationshipName'] = objProp.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about']
    for child in objProp:
        if child.tag=="{http://www.w3.org/2000/01/rdf-schema#}subPropertyOf":
            df_relationships.loc[len(df_relationships)-1,'subPropertyOf'] = child.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource"]
        if child.tag=="{http://www.w3.org/2002/07/owl#}inverseOf":
            df_relationships.loc[len(df_relationships)-1,'inverseOf'] = child.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource"]
        
print(df_relationships)

                                  relationshipName  \
0            http://radlex.org/RID/Anatomical_Site   
1                http://radlex.org/RID/Anterior_to   
2                http://radlex.org/RID/Attaches_to   
3            http://radlex.org/RID/Blood_Supply_of   
4                 http://radlex.org/RID/Bounded_by   
5                     http://radlex.org/RID/Bounds   
6                  http://radlex.org/RID/Branch_Of   
7             http://radlex.org/RID/Branch_Part_of   
8     http://radlex.org/RID/Constitutional_Part_Of   
9               http://radlex.org/RID/Contained_In   
10                  http://radlex.org/RID/Contains   
11           http://radlex.org/RID/Continuous_With   
12                 http://radlex.org/RID/Distal_to   
13               http://radlex.org/RID/Drains_Into   
14               http://radlex.org/RID/External_to   
15          http://radlex.org/RID/Has_Blood_Supply   
16                http://radlex.org/RID/Has_Branch   
17           http://radlex.o

## A set of concept attributes

In [214]:
annotation_properties = root.findall("{http://www.w3.org/2002/07/owl#}AnnotationProperty")
df_labels = pd.DataFrame(columns=['labelName','subPropertyOf'])

for annProp in annotation_properties:
    df_labels.loc[len(df_labels),'labelName'] = annProp.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about']
    for child in annProp:
        if child.tag=="{http://www.w3.org/2000/01/rdf-schema#}subPropertyOf":
            df_labels.loc[len(df_labels)-1,'subPropertyOf'] = child.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource"]
            
print(df_labels)

                                            labelName  \
0      http://data.bioontology.org/metadata/prefixIRI   
1                           http://radlex.org/RID/AAL   
2                       http://radlex.org/RID/Acronym   
3                     http://radlex.org/RID/CMA_Label   
4         http://radlex.org/RID/Changes_made_to_class   
5                       http://radlex.org/RID/Comment   
6                       http://radlex.org/RID/Created   
7                    http://radlex.org/RID/Definition   
8                 http://radlex.org/RID/Domain_string   
9                         http://radlex.org/RID/FMAID   
10                   http://radlex.org/RID/Freesurfer   
11                   http://radlex.org/RID/JHU_DTI-81   
12  http://radlex.org/RID/JHU_White-Matter_Tractog...   
13          http://radlex.org/RID/Misspelling_of_term   
14  http://radlex.org/RID/Preferred_Name_for_Obsolete   
15               http://radlex.org/RID/Preferred_name   
16        http://radlex.org/RID

## A set of concepts

{http://radlex.org/RID/}Preferred_name
{http://www.w3.org/2002/07/owl#}onProperty
{http://www.w3.org/2000/01/rdf-schema#}subClassOf
{http://www.w3.org/2002/07/owl#}Class
{http://www.w3.org/1999/02/22-rdf-syntax-ns#}Description
{http://radlex.org/RID/}SNOMED_ID
{http://www.w3.org/2002/07/owl#}Restriction
{http://www.w3.org/2000/01/rdf-schema#}label
{http://radlex.org/RID/}Misspelling_of_term
{http://radlex.org/RID/}Comment
{http://radlex.org/RID/}Synonym
{http://radlex.org/RID/}Replaced_by
{http://radlex.org/RID/}Definition
{http://www.w3.org/2002/07/owl#}someValuesFrom
{http://radlex.org/RID/}UMLS_ID

In [233]:
def getAttribValue_resource(node:ET.Element):
    try:
        return node.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource']
    except KeyError:
        logger.error(f"Tag:[{node.tag}], attr:[{node.attrib}], text:[{node.text}]")
        raise KeyError
        
def getRelationship(node:ET.Element):
    # Relationship elements are surrounded by the "Restriction" label
    # The sub elements only contain two types: onProperty and someValuesFrom
    assert ele[0].tag == "{http://www.w3.org/2002/07/owl#}Restriction"
    for relationship_node in ele[0]:
        if relationship_node.tag == "{http://www.w3.org/2002/07/owl#}onProperty":
            relationship_label = getAttribValue_resource(relationship_node)
        elif relationship_node.tag == "{http://www.w3.org/2002/07/owl#}someValuesFrom":
            refers_to = getAttribValue_resource(relationship_node)
        else:
            logger.trace(f"Tag:[{relationship_node.tag}], attr:[{relationship_node.attrib}], text:[{relationship_node.text}]")
    return (relationship_label,refers_to)

In [234]:
classes = root.findall("{http://www.w3.org/2002/07/owl#}Class")
df_classes = pd.DataFrame(columns=['classID','preferredName','definition'])
df_class_relationship = pd.DataFrame(columns=['classID_1','relationship','classID_2'])
classList = []
hierarchyList = []
relationshipList = []
temp = set()

for class_node in classes:
    classItem = []
    classID = class_node.attrib['{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about']
    classItem.append(classItem)
    for ele in class_node.iter():
        if ele.tag == "{http://radlex.org/RID/}Preferred_name":
            classItem.append(ele.text)
        if ele.tag == "{http://www.w3.org/2000/01/rdf-schema#}subClassOf":
            if ele.attrib:
                borderClassID = getAttribValue_resource(ele)
                hierarchyList.append([classID,borderClassID])
            else:
                (relationship_label,refers_to) = getRelationship(ele)
                relationshipList.append([classID,relationship_label,refers_to])
                
    classList.append(classItem)
    
print(relationshipList)

# Class hierarchy table
# df_class_hierarchy = pd.DataFrame(hierarchyList, columns=['classID','subClassOf'])
# print(df_class_hierarchy)

# for i in hierarchyList:
#     print(i)

2022-06-18 23:48:51.361 | ERROR    | __main__:getAttribValue_resource:5 - Tag:[{http://www.w3.org/2002/07/owl#}someValuesFrom], attr:[{}], text:[
                    ]


KeyError: 